In [23]:
import nltk
from nltk.cluster.util import cosine_distance
from nltk.corpus import stopwords
import numpy as np
import networkx as nx

In [87]:
# To read the text from the file and remove non-alphabetic characters.

def read_article(filename):
    file=open(filename,"r")
    filedata=file.readlines()
    article=filedata[0].split(". ") # creates a list of individual sentences
    sen=[]
    for s in article:
        sen.append(s.replace("[^a-zA-Z]", " ").split(" ")) # split each sentence to list of words
    sen.pop() #remove the last element(probably invalid sentence)
    return sen



In [74]:
#To find the similarity between two sentenses

def sentence_simil(sent1,sent2,stopwords=None):
    if stopwords==None:
        stopwords=[]
    sent1=[w.lower() for w in sent1]
    sent2=[w.lower() for w in sent2]
    all_words=list(set(sent1+sent2))
    vect1=[0]*len(all_words)
    vect2=[0]*len(all_words)
    for w in sent1:
        if w in stopwords:
            continue
        vect1[all_words.index(w)]+=1
    for w in sent2:
        if w in stopwords:
            continue
        vect2[all_words.index(w)]+=1
    return 1-cosine_distance(vect1, vect2)




In [75]:
# Generate a similarity measure.

def gen_sim_matrix(sentences, stopwords):
    sim_matrix=np.zeros((len(sentences),len(sentences)))
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i==j :
                continue
            sim_matrix[i][j]=sentence_simil(sentences[i],sentences[j],stopwords)
    return sim_matrix



In [100]:


def gen_summary(filename, topn=5):
    stop_words=stopwords.words('english')
    summarize_text=[]
    sentences=read_article(filename)
    sen_sim=gen_sim_matrix(sentences, stop_words)
    sen_sim_graph=nx.from_numpy_array(sen_sim) # In the resulting graph, each sentence is represented as a node,
    #and the edges between nodes represent the similarity between sentences. The weight of each edge corresponds 
    # to the similarity score.
    scores=nx.pagerank(sen_sim_graph) #Each node is assigned a PageRank score, indicating its importance within the graph.
    ranked_sen=sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    for i in range(topn):
        summarize_text.append(" ".join(ranked_sen[i][1]))
    print("Summary: \n", ". ".join(summarize_text))
    

In [101]:
gen_summary("test.txt",3)

Summary: 
 This is where the data is processed for presentation in an easy-to-consume manner to the right audience for the right purpose. Data visualization is the last step in the data life cycle. The right visualization requires an understanding of the consumerâ€™s needs, nature of the data, and the many tools and techniques available to present data
